### Manipulation of Peaks de novo results of ETNP 2017 P2 samples LC-MS/MS data using python.

Starting with:

    Peaks de novo results (.csv) of PTM-optimized database searches > ALC 50%

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the running # (e.g., 233) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

We don't have technical duplicates here, sadly, unlike the MED4 Pro samples. I exported PeaksDN search results CSVs into my ETNP 2017 git repo:

In [2]:
cd /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/

/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt


In [3]:
ls

231/  233/  243/  273/  278/  378/


In [4]:
ls 278

ETNP_SKQ17_DENOVO_168_278-265m-trap_DN50_15ppm.csv
ETNP_SKQ17_DENOVO_168_278-265m-trap_DN50_15ppm.xml


In [5]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

## 278: 265 m +P filtered on 0.3 um GF-75

In [7]:
# read the CSV into a dataframe using the pandas read_csv function
peaks278_50 = pd.read_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/278/ETNP_SKQ17_DENOVO_168_278-265m-trap_DN50_15ppm.csv")

# also make a dataframe but keep only peptides >80% ALC
peaks278_80 = peaks278_50.loc[peaks278_50['ALC (%)'] >= 80].copy()

# how many de novo sequence candidates >50% ALC?
print("# redundant Peaks peptides >50% ALC in dataframe", len(peaks278_50))
# how many de novo sequence candidates >80ALC?
print("# redundant Peaks peptides >80% ALC in dataframe", len(peaks278_80))

#look at the dataframe
peaks278_50.head()

# redundant Peaks peptides >50% ALC in dataframe 10488
# redundant Peaks peptides >80% ALC in dataframe 1482


,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,Area,Mass,ppm,PTM,local confidence (%),tag (>=0%),mode
0,5,6054,20181004_4-19_265m_topP_278_90min.raw,WLEEDK,6,98,6,410.1956,2,21.80,920000.0,818.3810,-5.3,NaN,97 99 100 99 96 97,WLEEDK,CID
1,5,14925,20181004_4-19_265m_topP_278_90min.raw,M(+15.99)TAEELKK,8,98,8,483.2513,2,41.26,21600000.0,964.4899,-2.0,Oxidation (M),96 98 98 99 99 98 99 97,M(+15.99)TAEELKK,CID
2,5,13192,20181004_4-19_265m_topP_278_90min.raw,SC(+57.02)DWPK,6,97,6,396.6714,2,37.44,4220000.0,791.3272,1.3,Carbamidomethylation,98 99 99 97 96 97,SC(+57.02)DWPK,CID
3,5,16407,20181004_4-19_265m_topP_278_90min.raw,WNQ(+.98)QFLNAAK,10,97,10,610.7994,2,44.52,40600000.0,1219.5986,-11.7,Deamidation (NQ),95 98 99 96 98 98 96 99 99 98,WNQ(+.98)QFLNAAK,CID
4,5,6599,20181004_4-19_265m_topP_278_90min.raw,SEDVYAK,7,97,7,406.1959,2,22.94,831000.0,810.3759,1.6,NaN,95 99 99 96 97 98 98,SEDVYAK,CID


In [8]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks278_50['A'] = peaks278_50['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks278_50['C'] = peaks278_50['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks278_50['D'] = peaks278_50['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks278_50['E'] = peaks278_50['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks278_50['F'] = peaks278_50['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks278_50['G'] = peaks278_50['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks278_50['H'] = peaks278_50['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks278_50 output, there will be no isoleucines (they're lumped in with leucines)
peaks278_50['I'] = peaks278_50['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks278_50['K'] = peaks278_50['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks278_50['L'] = peaks278_50['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks278_50['M'] = peaks278_50['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks278_50['N'] = peaks278_50['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks278_50['P'] = peaks278_50['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks278_50['Q'] = peaks278_50['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks278_50['R'] = peaks278_50['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks278_50['S'] = peaks278_50['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks278_50['T'] = peaks278_50['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks278_50['V'] = peaks278_50['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks278_50['W'] = peaks278_50['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks278_50['Y'] = peaks278_50['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks278_50['c-carb'] = peaks278_50['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks278_50['m-oxid'] = peaks278_50['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks278_50['n-deam'] = peaks278_50['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks278_50['q-deam'] = peaks278_50['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# use a count function to enumerate the # of hydroxylated K's in each peptide
peaks278_50['k-hydr'] = peaks278_50['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of hydroxylated P's in each peptide
# no more P hydroxylation in final searches
# peaks278_50['p-hydr'] = peaks278_50['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaks278_50['r-meth'] = peaks278_50['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# create a column with 'stripped' peptide sequences using strip
peaks278_50['stripped peptide'] = peaks278_50['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks278_50['stripped length'] = peaks278_50['stripped peptide'].apply(len)

# total the number of modifications in sequence
peaks278_50['ptm-total'] = peaks278_50['c-carb'] + peaks278_50['m-oxid'] + peaks278_50['n-deam'] \
+ peaks278_50['q-deam'] + peaks278_50['k-hydr'] + peaks278_50['r-meth']

# calculate NAAF numerator for each peptide k
peaks278_50['NAAF num.'] = peaks278_50['Area'] / peaks278_50['stripped length']

# write modified dataframe to new csv file
peaks278_50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/278/ETNP-SKQ17-278-265m-trap_PTMopt_15ppm_DN50.csv")


# check out the results
peaks278_50.head()

,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,...,c-carb,m-oxid,n-deam,q-deam,k-hydr,r-meth,stripped peptide,stripped length,ptm-total,NAAF num.
0,5,6054,20181004_4-19_265m_topP_278_90min.raw,WLEEDK,6,98,6,410.1956,2,21.80,...,0,0,0,0,0,0,WLEEDK,6,0,1.533333e+05
1,5,14925,20181004_4-19_265m_topP_278_90min.raw,M(+15.99)TAEELKK,8,98,8,483.2513,2,41.26,...,0,1,0,0,0,0,MTAEELKK,8,1,2.700000e+06
2,5,13192,20181004_4-19_265m_topP_278_90min.raw,SC(+57.02)DWPK,6,97,6,396.6714,2,37.44,...,1,0,0,0,0,0,SCDWPK,6,1,7.033333e+05
3,5,16407,20181004_4-19_265m_topP_278_90min.raw,WNQ(+.98)QFLNAAK,10,97,10,610.7994,2,44.52,...,0,0,0,1,0,0,WNQQFLNAAK,10,1,4.060000e+06
4,5,6599,20181004_4-19_265m_topP_278_90min.raw,SEDVYAK,7,97,7,406.1959,2,22.94,...,0,0,0,0,0,0,SEDVYAK,7,0,1.187143e+05


In [9]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks278_50['A'].sum(),
        'C': peaks278_50['C'].sum(),
        'D': peaks278_50['D'].sum(),
        'E': peaks278_50['E'].sum(),
        'F': peaks278_50['F'].sum(),
        'G': peaks278_50['G'].sum(),
        'H': peaks278_50['H'].sum(),
        'I': peaks278_50['I'].sum(),
        'K': peaks278_50['K'].sum(),
        'L': peaks278_50['L'].sum(),
        'M': peaks278_50['M'].sum(),
        'N': peaks278_50['N'].sum(),
        'P': peaks278_50['P'].sum(),
        'Q': peaks278_50['Q'].sum(),
        'R': peaks278_50['R'].sum(),
        'S': peaks278_50['S'].sum(),
        'T': peaks278_50['T'].sum(),
        'V': peaks278_50['V'].sum(),
        'W': peaks278_50['W'].sum(),
        'Y': peaks278_50['Y'].sum(),
        'c-carb': peaks278_50['c-carb'].sum(),
        'm-oxid': peaks278_50['m-oxid'].sum(),
        'n-deam': peaks278_50['n-deam'].sum(),
        'q-deam': peaks278_50['q-deam'].sum(),
        'k-hydr': peaks278_50['k-hydr'].sum(),
        'r-meth': peaks278_50['r-meth'].sum(),
        'Total area': peaks278_50['Area'].sum(),
        'Total length': peaks278_50['stripped length'].sum()
       }

totalpeaks278_50 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',\
                                               'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', \
                                               'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'r-meth', \
                                               'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks278_50['% C w/ carb'] = totalpeaks278_50['c-carb'] / totalpeaks278_50['C'] 

# calculate percentage of M's that are oxidized
totalpeaks278_50['% M w/ oxid'] = totalpeaks278_50['m-oxid'] / totalpeaks278_50['M'] 

# calculate percentage of N's that are deamidated
totalpeaks278_50['% N w/ deam'] = totalpeaks278_50['n-deam'] / totalpeaks278_50['N'] 

# calculate percentage of Q's that are deamidated
totalpeaks278_50['% Q w/ deam'] = totalpeaks278_50['q-deam'] / totalpeaks278_50['Q'] 

# calculate percentage of K's that are hydroxylated
totalpeaks278_50['% K w/ hydr'] = totalpeaks278_50['k-hydr'] / totalpeaks278_50['K'] 

# calculate percentage of P's that are hydroxylated
#totalpeaks278_50['% P w/ hydr'] = totalpeaks278_50['p-hydr'] / totalpeaks278_50['K'] 

# calculate percentage of R's that are methylated
totalpeaks278_50['% R w/ meth'] = totalpeaks278_50['r-meth'] / totalpeaks278_50['R'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks278_50['NAAF denom.'] = totalpeaks278_50['Total area'] / totalpeaks278_50['Total length']

# write modified dataframe to new txt file
totalpeaks278_50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/278/ETNP-SKQ17-278-265m-trap_PTMopt_15ppm_DN50_totals.csv")

totalpeaks278_50.head()

,A,C,D,E,F,G,H,I,K,L,...,r-meth,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,% K w/ hydr,% R w/ meth,NAAF denom.
sample total,5564,1919,5229,6154,2787,4374,2690,0,9598,9028,...,2975,1.785712e+10,80491,1.0,0.428571,0.236156,0.026813,0.209419,0.479607,221852.426966


In [10]:
# use the calculated NAAF factor (in totalpeaks278 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we will with Comet results

NAAF50 = 221852.426966

# use NAAF >50% ALC to get NAAF
peaks278_50['NAAF factor'] = (peaks278_50['NAAF num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaks278_AA50 = peaks278_50[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                             'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', \
                             'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'r-meth']].copy()

# multiply the NAAF50 factor by the AAs to normalize its abundance by peak area and peptide length

peaks278_AA50['A-NAAF50'] = peaks278_AA50['A'] * peaks278_50['NAAF factor']
peaks278_AA50['C-NAAF50'] = peaks278_AA50['C'] * peaks278_50['NAAF factor']
peaks278_AA50['D-NAAF50'] = peaks278_AA50['D'] * peaks278_50['NAAF factor']
peaks278_AA50['E-NAAF50'] = peaks278_AA50['E'] * peaks278_50['NAAF factor']
peaks278_AA50['F-NAAF50'] = peaks278_AA50['F'] * peaks278_50['NAAF factor']
peaks278_AA50['G-NAAF50'] = peaks278_AA50['G'] * peaks278_50['NAAF factor']
peaks278_AA50['H-NAAF50'] = peaks278_AA50['H'] * peaks278_50['NAAF factor']
peaks278_AA50['I-NAAF50'] = peaks278_AA50['I'] * peaks278_50['NAAF factor']
peaks278_AA50['K-NAAF50'] = peaks278_AA50['K'] * peaks278_50['NAAF factor']
peaks278_AA50['L-NAAF50'] = peaks278_AA50['L'] * peaks278_50['NAAF factor']
peaks278_AA50['M-NAAF50'] = peaks278_AA50['M'] * peaks278_50['NAAF factor']
peaks278_AA50['N-NAAF50'] = peaks278_AA50['N'] * peaks278_50['NAAF factor']
peaks278_AA50['P-NAAF50'] = peaks278_AA50['P'] * peaks278_50['NAAF factor']
peaks278_AA50['Q-NAAF50'] = peaks278_AA50['Q'] * peaks278_50['NAAF factor']
peaks278_AA50['R-NAAF50'] = peaks278_AA50['R'] * peaks278_50['NAAF factor']
peaks278_AA50['S-NAAF50'] = peaks278_AA50['S'] * peaks278_50['NAAF factor']
peaks278_AA50['T-NAAF50'] = peaks278_AA50['T'] * peaks278_50['NAAF factor']
peaks278_AA50['V-NAAF50'] = peaks278_AA50['V'] * peaks278_50['NAAF factor']
peaks278_AA50['W-NAAF50'] = peaks278_AA50['W'] * peaks278_50['NAAF factor']
peaks278_AA50['Y-NAAF50'] = peaks278_AA50['Y'] * peaks278_50['NAAF factor']

# multiply the NAAF50 factor by the PTMs normalize its abundance by peak area and peptide length

peaks278_AA50['ccarb-NAAF50'] = peaks278_AA50['c-carb'] * peaks278_AA50['NAAF factor']
peaks278_AA50['moxid-NAAF50'] = peaks278_AA50['m-oxid'] * peaks278_AA50['NAAF factor']
peaks278_AA50['ndeam-NAAF50'] = peaks278_AA50['n-deam'] * peaks278_AA50['NAAF factor']
peaks278_AA50['qdeam-NAAF50'] = peaks278_AA50['q-deam'] * peaks278_AA50['NAAF factor']
peaks278_AA50['khydr-NAAF50'] = peaks278_AA50['k-hydr'] * peaks278_AA50['NAAF factor']
peaks278_AA50['rmeth-NAAF50'] = peaks278_AA50['r-meth'] * peaks278_AA50['NAAF factor']

# write the dataframe to a new csv
peaks278_AA50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/278/ETNP-SKQ17-278-265m-trap_PTMopt_15ppm_DN50_NAAF.csv")

peaks278_AA50.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,I,...,T-NAAF50,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50,khydr-NAAF50,rmeth-NAAF50
0,WLEEDK,0.691150,0,0,1,2,0,0,0,0,...,0.000000,0.000000,0.691150,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
1,MTAEELKK,12.170252,1,0,0,2,0,0,0,0,...,12.170252,0.000000,0.000000,0.000000,0.000000,12.170252,0.0,0.000000,0.0,0.0
2,SCDWPK,3.170276,0,1,1,0,0,0,0,0,...,0.000000,0.000000,3.170276,0.000000,3.170276,0.000000,0.0,0.000000,0.0,0.0
3,WNQQFLNAAK,18.300453,2,0,0,0,1,0,0,0,...,0.000000,0.000000,18.300453,0.000000,0.000000,0.000000,0.0,18.300453,0.0,0.0
4,SEDVYAK,0.535105,1,0,1,1,0,0,0,0,...,0.000000,0.535105,0.000000,0.535105,0.000000,0.000000,0.0,0.000000,0.0,0.0


In [11]:
# made a new dataframe that contains the sums of NAAF normalized AAs for comet278 PeaksDN results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaks278_AA50['NAAF factor'].sum(),
        'A-NAAF': peaks278_AA50['A-NAAF50'].sum(),
        'C-NAAF': peaks278_AA50['C-NAAF50'].sum(),
        'D-NAAF': peaks278_AA50['D-NAAF50'].sum(),
        'E-NAAF': peaks278_AA50['E-NAAF50'].sum(),
        'F-NAAF': peaks278_AA50['F-NAAF50'].sum(),
        'G-NAAF': peaks278_AA50['G-NAAF50'].sum(),
        'H-NAAF': peaks278_AA50['H-NAAF50'].sum(),
        'I-NAAF': peaks278_AA50['I-NAAF50'].sum(),
        'K-NAAF': peaks278_AA50['K-NAAF50'].sum(),
        'L-NAAF': peaks278_AA50['L-NAAF50'].sum(),
        'M-NAAF': peaks278_AA50['M-NAAF50'].sum(),
        'N-NAAF': peaks278_AA50['N-NAAF50'].sum(),
        'P-NAAF': peaks278_AA50['P-NAAF50'].sum(),
        'Q-NAAF': peaks278_AA50['Q-NAAF50'].sum(),
        'R-NAAF': peaks278_AA50['R-NAAF50'].sum(),
        'S-NAAF': peaks278_AA50['S-NAAF50'].sum(),
        'T-NAAF': peaks278_AA50['T-NAAF50'].sum(),
        'V-NAAF': peaks278_AA50['V-NAAF50'].sum(),
        'W-NAAF': peaks278_AA50['W-NAAF50'].sum(),
        'Y-NAAF': peaks278_AA50['Y-NAAF50'].sum(),
        'C-carb-NAAF': peaks278_AA50['ccarb-NAAF50'].sum(),
        'M-oxid-NAAF': peaks278_AA50['moxid-NAAF50'].sum(),
        'N-deam-NAAF': peaks278_AA50['ndeam-NAAF50'].sum(),
        'Q-deam-NAAF': peaks278_AA50['qdeam-NAAF50'].sum(),
        'K-hydr-NAAF': peaks278_AA50['khydr-NAAF50'].sum(),
        'R-meth-NAAF': peaks278_AA50['rmeth-NAAF50'].sum()
       }

totalpeaks278_AA50 = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF', 'K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF', 'K-hydr-NAAF',\
                                                   'R-meth-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, P, and Rs


totalpeaks278_AA50['% C w/ carb. NAAF'] = totalpeaks278_AA50['C-carb-NAAF'] / totalpeaks278_AA50['C-NAAF']
totalpeaks278_AA50['% M w/ oxid. NAAF'] = totalpeaks278_AA50['M-oxid-NAAF'] / totalpeaks278_AA50['M-NAAF']
totalpeaks278_AA50['% N w/ deam. NAAF'] = totalpeaks278_AA50['N-deam-NAAF'] / totalpeaks278_AA50['N-NAAF']
totalpeaks278_AA50['% Q w/ deam. NAAF'] = totalpeaks278_AA50['Q-deam-NAAF'] / totalpeaks278_AA50['Q-NAAF']
totalpeaks278_AA50['% K w/ hydr. NAAF'] = totalpeaks278_AA50['K-hydr-NAAF'] / totalpeaks278_AA50['K-NAAF']
totalpeaks278_AA50['% R w/ meth. NAAF'] = totalpeaks278_AA50['R-meth-NAAF'] / totalpeaks278_AA50['R-NAAF']

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks278_AA50['NAAF check'] = totalpeaks278_AA50['NAAF'] / 221852.426966

# write modified dataframe to new txt file, same name + totals
totalpeaks278_AA50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/278/ETNP-SKQ17-278-265m-trap_PTMopt_DN50_NAAF_totals.csv")

totalpeaks278_AA50.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,Q-deam-NAAF,K-hydr-NAAF,R-meth-NAAF,% C w/ carb. NAAF,% M w/ oxid. NAAF,% N w/ deam. NAAF,% Q w/ deam. NAAF,% K w/ hydr. NAAF,% R w/ meth. NAAF,NAAF check
sample total,12698.861516,7206.423438,1977.929736,5780.432065,6519.167339,2847.316116,5307.700617,2910.267226,0.0,10063.583942,...,130.427599,3681.705649,4244.306896,1.0,0.481779,0.312771,0.056863,0.365844,0.530163,0.05724


### Same process but for de novo peptide >80 % ALC:

In [12]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks278_80['A'] = peaks278_80['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks278_80['C'] = peaks278_80['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks278_80['D'] = peaks278_80['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks278_80['E'] = peaks278_80['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks278_80['F'] = peaks278_80['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks278_80['G'] = peaks278_80['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks278_80['H'] = peaks278_80['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks278_80 output, there will be no isoleucines (they're lumped in with leucines)
peaks278_80['I'] = peaks278_80['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks278_80['K'] = peaks278_80['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks278_80['L'] = peaks278_80['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks278_80['M'] = peaks278_80['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks278_80['N'] = peaks278_80['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks278_80['P'] = peaks278_80['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks278_80['Q'] = peaks278_80['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks278_80['R'] = peaks278_80['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks278_80['S'] = peaks278_80['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks278_80['T'] = peaks278_80['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks278_80['V'] = peaks278_80['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks278_80['W'] = peaks278_80['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks278_80['Y'] = peaks278_80['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks278_80['c-carb'] = peaks278_80['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks278_80['m-oxid'] = peaks278_80['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks278_80['n-deam'] = peaks278_80['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks278_80['q-deam'] = peaks278_80['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# use a count function to enumerate the # of hydroxylated K's in each peptide
peaks278_80['k-hydr'] = peaks278_80['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of hydroxylated P's in each peptide
#peaks278_80['p-hydr'] = peaks278_80['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaks278_80['r-meth'] = peaks278_80['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# create a column with 'stripped' peptide sequences using strip
peaks278_80['stripped peptide'] = peaks278_80['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks278_80['stripped length'] = peaks278_80['stripped peptide'].apply(len)

# total the number of modifications in sequence
peaks278_80['ptm-total'] = peaks278_80['c-carb'] + peaks278_80['m-oxid'] + peaks278_80['n-deam'] + \
peaks278_80['q-deam'] + peaks278_80['k-hydr'] + peaks278_80['r-meth']

# calculate NAAF numerator for each peptide k
peaks278_80['NAAF num.'] = peaks278_80['Area'] / peaks278_80['stripped length']

# write modified dataframe to new csv file
peaks278_80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/278/ETNP-SKQ17-278-265m-trap_PTMopt_15ppm_DN80.csv")

# check out the results
peaks278_80.head()

,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,...,c-carb,m-oxid,n-deam,q-deam,k-hydr,r-meth,stripped peptide,stripped length,ptm-total,NAAF num.
0,5,6054,20181004_4-19_265m_topP_278_90min.raw,WLEEDK,6,98,6,410.1956,2,21.80,...,0,0,0,0,0,0,WLEEDK,6,0,1.533333e+05
1,5,14925,20181004_4-19_265m_topP_278_90min.raw,M(+15.99)TAEELKK,8,98,8,483.2513,2,41.26,...,0,1,0,0,0,0,MTAEELKK,8,1,2.700000e+06
2,5,13192,20181004_4-19_265m_topP_278_90min.raw,SC(+57.02)DWPK,6,97,6,396.6714,2,37.44,...,1,0,0,0,0,0,SCDWPK,6,1,7.033333e+05
3,5,16407,20181004_4-19_265m_topP_278_90min.raw,WNQ(+.98)QFLNAAK,10,97,10,610.7994,2,44.52,...,0,0,0,1,0,0,WNQQFLNAAK,10,1,4.060000e+06
4,5,6599,20181004_4-19_265m_topP_278_90min.raw,SEDVYAK,7,97,7,406.1959,2,22.94,...,0,0,0,0,0,0,SEDVYAK,7,0,1.187143e+05


In [13]:
# made a new dataframe that contains the sums of certain columns in modified 
#peptide dataframe above (for >80% ALC)

index = ['sample total']

data = {'A': peaks278_80['A'].sum(),
        'C': peaks278_80['C'].sum(),
        'D': peaks278_80['D'].sum(),
        'E': peaks278_80['E'].sum(),
        'F': peaks278_80['F'].sum(),
        'G': peaks278_80['G'].sum(),
        'H': peaks278_80['H'].sum(),
        'I': peaks278_80['I'].sum(),
        'K': peaks278_80['K'].sum(),
        'L': peaks278_80['L'].sum(),
        'M': peaks278_80['M'].sum(),
        'N': peaks278_80['N'].sum(),
        'P': peaks278_80['P'].sum(),
        'Q': peaks278_80['Q'].sum(),
        'R': peaks278_80['R'].sum(),
        'S': peaks278_80['S'].sum(),
        'T': peaks278_80['T'].sum(),
        'V': peaks278_80['V'].sum(),
        'W': peaks278_80['W'].sum(),
        'Y': peaks278_80['Y'].sum(),
        'c-carb': peaks278_80['c-carb'].sum(),
        'm-oxid': peaks278_80['m-oxid'].sum(),
        'n-deam': peaks278_80['n-deam'].sum(),
        'q-deam': peaks278_80['q-deam'].sum(),
        'k-hydr': peaks278_80['k-hydr'].sum(),
        'r-meth': peaks278_80['r-meth'].sum(),
        'Total area': peaks278_80['Area'].sum(),
        'Total length': peaks278_80['stripped length'].sum()
       }

totalpeaks278_80 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M',\
                                               'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb',\
                                               'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'r-meth',\
                                               'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks278_80['% C w/ carb'] = totalpeaks278_80['c-carb'] / totalpeaks278_80['C'] 

# calculate percentage of M's that are oxidized
totalpeaks278_80['% M w/ oxid'] = totalpeaks278_80['m-oxid'] / totalpeaks278_80['M'] 

# calculate percentage of N's that are deamidated
totalpeaks278_80['% N w/ deam'] = totalpeaks278_80['n-deam'] / totalpeaks278_80['N'] 

# calculate percentage of Q's that are deamidated
totalpeaks278_80['% Q w/ deam'] = totalpeaks278_80['q-deam'] / totalpeaks278_80['Q'] 

# calculate percentage of K's that are hydroxylated
totalpeaks278_80['% K w/ hydr'] = totalpeaks278_80['k-hydr'] / totalpeaks278_80['K'] 

# calculate percentage of P's that are hydroxylated
#totalpeaks278_80['% P w/ hydr'] = totalpeaks278_80['p-hydr'] / totalpeaks278_80['K'] 

# calculate percentage of R's that are methylated
totalpeaks278_80['% R w/ meth'] = totalpeaks278_80['r-meth'] / totalpeaks278_80['R'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks278_80['NAAF denom.'] = totalpeaks278_80['Total area'] / totalpeaks278_80['Total length']

# write modified dataframe to new txt file
totalpeaks278_80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/278/ETNP-SKQ17-278-265m-trap_PTMopt_15ppm_DN80_totals.csv")

totalpeaks278_80.head()

,A,C,D,E,F,G,H,I,K,L,...,r-meth,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,% K w/ hydr,% R w/ meth,NAAF denom.
sample total,887,169,785,1286,385,557,167,0,1173,1654,...,255,5.529521e+09,11548,1.0,0.483696,0.331516,0.071053,0.094629,0.328608,478829.326697


In [14]:
# use the calculated NAAF factor (in totalpeaks278 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we will with Comet results

NAAF80 = 478829.326697

# use NAAF >80% ALC to get NAAF
peaks278_80['NAAF factor'] = (peaks278_80['NAAF num.'])/NAAF80

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaks278_AA80 = peaks278_80[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                             'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                             'n-deam', 'q-deam', 'k-hydr', 'r-meth']].copy()

# multiply the NAAF80 factor by the AAs to normalize its abundance by peak area and peptide length

peaks278_AA80['A-NAAF80'] = peaks278_AA80['A'] * peaks278_80['NAAF factor']
peaks278_AA80['C-NAAF80'] = peaks278_AA80['C'] * peaks278_80['NAAF factor']
peaks278_AA80['D-NAAF80'] = peaks278_AA80['D'] * peaks278_80['NAAF factor']
peaks278_AA80['E-NAAF80'] = peaks278_AA80['E'] * peaks278_80['NAAF factor']
peaks278_AA80['F-NAAF80'] = peaks278_AA80['F'] * peaks278_80['NAAF factor']
peaks278_AA80['G-NAAF80'] = peaks278_AA80['G'] * peaks278_80['NAAF factor']
peaks278_AA80['H-NAAF80'] = peaks278_AA80['H'] * peaks278_80['NAAF factor']
peaks278_AA80['I-NAAF80'] = peaks278_AA80['I'] * peaks278_80['NAAF factor']
peaks278_AA80['K-NAAF80'] = peaks278_AA80['K'] * peaks278_80['NAAF factor']
peaks278_AA80['L-NAAF80'] = peaks278_AA80['L'] * peaks278_80['NAAF factor']
peaks278_AA80['M-NAAF80'] = peaks278_AA80['M'] * peaks278_80['NAAF factor']
peaks278_AA80['N-NAAF80'] = peaks278_AA80['N'] * peaks278_80['NAAF factor']
peaks278_AA80['P-NAAF80'] = peaks278_AA80['P'] * peaks278_80['NAAF factor']
peaks278_AA80['Q-NAAF80'] = peaks278_AA80['Q'] * peaks278_80['NAAF factor']
peaks278_AA80['R-NAAF80'] = peaks278_AA80['R'] * peaks278_80['NAAF factor']
peaks278_AA80['S-NAAF80'] = peaks278_AA80['S'] * peaks278_80['NAAF factor']
peaks278_AA80['T-NAAF80'] = peaks278_AA80['T'] * peaks278_80['NAAF factor']
peaks278_AA80['V-NAAF80'] = peaks278_AA80['V'] * peaks278_80['NAAF factor']
peaks278_AA80['W-NAAF80'] = peaks278_AA80['W'] * peaks278_80['NAAF factor']
peaks278_AA80['Y-NAAF80'] = peaks278_AA80['Y'] * peaks278_80['NAAF factor']

# multiply the NAAF80 factor by the PTMs normalize its abundance by peak area and peptide length

peaks278_AA80['ccarb-NAAF80'] = peaks278_AA80['c-carb'] * peaks278_AA80['NAAF factor']
peaks278_AA80['moxid-NAAF80'] = peaks278_AA80['m-oxid'] * peaks278_AA80['NAAF factor']
peaks278_AA80['ndeam-NAAF80'] = peaks278_AA80['n-deam'] * peaks278_AA80['NAAF factor']
peaks278_AA80['qdeam-NAAF80'] = peaks278_AA80['q-deam'] * peaks278_AA80['NAAF factor']
peaks278_AA80['khydr-NAAF80'] = peaks278_AA80['k-hydr'] * peaks278_AA80['NAAF factor']
peaks278_AA80['rmeth-NAAF80'] = peaks278_AA80['r-meth'] * peaks278_AA80['NAAF factor']

# write the dataframe to a new csv
peaks278_AA80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/278/ETNP-SKQ17-278-265m-trap_PTMopt_15ppm_DN80_NAAF.csv")

peaks278_AA80.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,I,...,T-NAAF80,V-NAAF80,W-NAAF80,Y-NAAF80,ccarb-NAAF80,moxid-NAAF80,ndeam-NAAF80,qdeam-NAAF80,khydr-NAAF80,rmeth-NAAF80
0,WLEEDK,0.320225,0,0,1,2,0,0,0,0,...,0.000000,0.000000,0.320225,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.0
1,MTAEELKK,5.638752,1,0,0,2,0,0,0,0,...,5.638752,0.000000,0.000000,0.000000,0.00000,5.638752,0.0,0.000000,0.0,0.0
2,SCDWPK,1.468860,0,1,1,0,0,0,0,0,...,0.000000,0.000000,1.468860,0.000000,1.46886,0.000000,0.0,0.000000,0.0,0.0
3,WNQQFLNAAK,8.479013,2,0,0,0,1,0,0,0,...,0.000000,0.000000,8.479013,0.000000,0.00000,0.000000,0.0,8.479013,0.0,0.0
4,SEDVYAK,0.247926,1,0,1,1,0,0,0,0,...,0.000000,0.247926,0.000000,0.247926,0.00000,0.000000,0.0,0.000000,0.0,0.0


In [15]:
# made a new dataframe that contains the sums of NAAF normalized AAs for comet278 PeaksDN results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaks278_AA80['NAAF factor'].sum(),
        'A-NAAF': peaks278_AA80['A-NAAF80'].sum(),
        'C-NAAF': peaks278_AA80['C-NAAF80'].sum(),
        'D-NAAF': peaks278_AA80['D-NAAF80'].sum(),
        'E-NAAF': peaks278_AA80['E-NAAF80'].sum(),
        'F-NAAF': peaks278_AA80['F-NAAF80'].sum(),
        'G-NAAF': peaks278_AA80['G-NAAF80'].sum(),
        'H-NAAF': peaks278_AA80['H-NAAF80'].sum(),
        'I-NAAF': peaks278_AA80['I-NAAF80'].sum(),
        'K-NAAF': peaks278_AA80['K-NAAF80'].sum(),
        'L-NAAF': peaks278_AA80['L-NAAF80'].sum(),
        'M-NAAF': peaks278_AA80['M-NAAF80'].sum(),
        'N-NAAF': peaks278_AA80['N-NAAF80'].sum(),
        'P-NAAF': peaks278_AA80['P-NAAF80'].sum(),
        'Q-NAAF': peaks278_AA80['Q-NAAF80'].sum(),
        'R-NAAF': peaks278_AA80['R-NAAF80'].sum(),
        'S-NAAF': peaks278_AA80['S-NAAF80'].sum(),
        'T-NAAF': peaks278_AA80['T-NAAF80'].sum(),
        'V-NAAF': peaks278_AA80['V-NAAF80'].sum(),
        'W-NAAF': peaks278_AA80['W-NAAF80'].sum(),
        'Y-NAAF': peaks278_AA80['Y-NAAF80'].sum(),
        'C-carb-NAAF': peaks278_AA80['ccarb-NAAF80'].sum(),
        'M-oxid-NAAF': peaks278_AA80['moxid-NAAF80'].sum(),
        'N-deam-NAAF': peaks278_AA80['ndeam-NAAF80'].sum(),
        'Q-deam-NAAF': peaks278_AA80['qdeam-NAAF80'].sum(),
        'K-hydr-NAAF': peaks278_AA80['khydr-NAAF80'].sum(),
        'R-meth-NAAF': peaks278_AA80['rmeth-NAAF80'].sum()
       }

totalpeaks278_AA80 = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF', 'K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF', 'K-hydr-NAAF',\
                                                   'R-meth-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, P, and Rs


totalpeaks278_AA80['% C w/ carb. NAAF'] = totalpeaks278_AA80['C-carb-NAAF'] / totalpeaks278_AA80['C-NAAF']
totalpeaks278_AA80['% M w/ oxid. NAAF'] = totalpeaks278_AA80['M-oxid-NAAF'] / totalpeaks278_AA80['M-NAAF']
totalpeaks278_AA80['% N w/ deam. NAAF'] = totalpeaks278_AA80['N-deam-NAAF'] / totalpeaks278_AA80['N-NAAF']
totalpeaks278_AA80['% Q w/ deam. NAAF'] = totalpeaks278_AA80['Q-deam-NAAF'] / totalpeaks278_AA80['Q-NAAF']
totalpeaks278_AA80['% K w/ hydr. NAAF'] = totalpeaks278_AA80['K-hydr-NAAF'] / totalpeaks278_AA80['K-NAAF']
totalpeaks278_AA80['% R w/ meth. NAAF'] = totalpeaks278_AA80['R-meth-NAAF'] / totalpeaks278_AA80['R-NAAF']

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks278_AA80['NAAF check'] = totalpeaks278_AA80['NAAF'] / 478829.326697

# write modified dataframe to new txt file, same name + totals
totalpeaks278_AA80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/278/ETNP-SKQ17-278-265m-trap_PTMopt_15ppm_DN80_NAAF_totals.csv")

totalpeaks278_AA80.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,Q-deam-NAAF,K-hydr-NAAF,R-meth-NAAF,% C w/ carb. NAAF,% M w/ oxid. NAAF,% N w/ deam. NAAF,% Q w/ deam. NAAF,% K w/ hydr. NAAF,% R w/ meth. NAAF,NAAF check
sample total,1529.318041,1082.98612,129.771677,625.052273,1109.793016,280.219158,592.252388,125.425359,0.0,957.8277,...,22.696391,178.812916,355.837166,1.0,0.668331,0.377249,0.088905,0.186686,0.374069,0.003194


### Visualizing the results

In [16]:
print("ALC max: ", peaks278_80['ALC (%)'].max())
print("ALC min: ", peaks278_80['ALC (%)'].min())

ALC max:  98
ALC min:  80


### Exporting txt files of stripped peptides at confidence cutoffs:

In [17]:
# keep only peptide column >50% ALC
pep278_50_moddup = peaks278_50[["Peptide"]]

# keep only the stripped peptide column for the >50% ALC
# this is what we'll use for UniPept input, etc
pep278_50_dup = peaks278_50[["stripped peptide"]]

# deduplicate both of these lists
pep278_50_mod = pep278_50_moddup.drop_duplicates()
pep278_50 = pep278_50_dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified peptides in 278:", len(pep278_50_moddup))
print("Deduplicated modified peptides in 278:", len(pep278_50_mod))
print("Total strippled peptides in 278:", len(pep278_50_dup))
print("Deduplicated stripped peptides in 278:", len(pep278_50))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep278_50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/278/ETNP-SKQ17-278-265m-trap_PTMopt_15ppm_DN50_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 

!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/278/ETNP-SKQ17-278-265m-trap_PTMopt_15ppm_DN50_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/278/ETNP-SKQ17-278-265m-trap_PTMopt_15ppm_DN50_stripped_peptides.fas


Total modified peptides in 278: 10488
Deduplicated modified peptides in 278: 10424
Total strippled peptides in 278: 10488
Deduplicated stripped peptides in 278: 9735


In [18]:
# keep only peptide column >80% ALC
pep278_80_moddup = peaks278_80[["Peptide"]]

# keep only the stripped peptide column for the >80% ALC
# this is what we'll use for UniPept input, etc
pep278_80_dup = peaks278_80[["stripped peptide"]]

# deduplicate both of these lists
pep278_80_mod = pep278_80_moddup.drop_duplicates()
pep278_80 = pep278_80_dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified peptides in 278:", len(pep278_80_moddup))
print("Deduplicated modified peptides in 278:", len(pep278_80_mod))
print("Total strippled peptides in 278:", len(pep278_80_dup))
print("Deduplicated stripped peptides in 278:", len(pep278_80))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep278_80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/278/ETNP-SKQ17-278-265m-trap_PTMopt_15ppm_DN80_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 

!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/278/ETNP-SKQ17-278-265m-trap_PTMopt_15ppm_DN80_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/278/ETNP-SKQ17-278-265m-trap_PTMopt_15ppm_DN80_stripped_peptides.fas


Total modified peptides in 278: 1482
Deduplicated modified peptides in 278: 1463
Total strippled peptides in 278: 1482
Deduplicated stripped peptides in 278: 1441
